### Data_Preprocessing

#### Sentiment Analysis with Mobile review data from Amazon https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones

In [3]:
#packages imports

from tqdm import tqdm
import pandas as pd

import re, string

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aishwarya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [1]:
#convert jupyter notebook into html file
!jupyter nbconvert --to html Data_Preprocessing.ipynb

[NbConvertApp] Converting notebook Data_Preprocessing.ipynb to html
[NbConvertApp] Writing 313284 bytes to Data_Preprocessing.html


In [5]:
#reading the data file from the csv file using pandas

original = pd.read_csv(r"C:\Users\Aishwarya\PROJECTS\NLP\main\Amazon_Unlocked_Mobile.csv")
original.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [6]:
# dropping the price and the review votes fields
original.drop('Price', axis=1, inplace=True)
original.drop('Review Votes', axis=1, inplace=True)


In [7]:
original.drop('Product Name', axis=1, inplace=True)
original.drop('Brand Name', axis=1, inplace=True)

In [8]:
# displaying the details of the dataframe
original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413840 entries, 0 to 413839
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Rating   413840 non-null  int64 
 1   Reviews  413778 non-null  object
dtypes: int64(1), object(1)
memory usage: 6.3+ MB


In [9]:
# displaying rows with NaN or null values
incomplete_rows = original[original.isnull().any(axis = 1)]
print(incomplete_rows.shape)
incomplete_rows.head()

(62, 2)


,Rating,Reviews
6428,4,NaN
12533,1,NaN
13024,4,NaN
13533,5,NaN
14651,4,NaN


In [10]:
# dropping all the rows with null values
original.dropna(inplace=True)

In [11]:
# displaying the details of the dataframe
original.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 413778 entries, 0 to 413839
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Rating   413778 non-null  int64 
 1   Reviews  413778 non-null  object
dtypes: int64(1), object(1)
memory usage: 9.5+ MB


In [12]:
original.head(10)

,Rating,Reviews
0,5,I feel so LUCKY to have found this used (phone...
1,4,"nice phone, nice up grade from my pantach revu..."
2,5,Very pleased
3,4,It works good but it goes slow sometimes but i...
4,4,Great phone to replace my lost phone. The only...
5,1,I already had a phone with problems... I know ...
6,2,The charging port was loose. I got that solder...
7,2,"Phone looks good but wouldn't stay charged, ha..."
8,5,I originally was using the Samsung S2 Galaxy f...
9,3,It's battery life is great. It's very responsi...


In [13]:
# Displaying the first 5 rows of 'review text' field
original['Reviews'].head(5)

0    I feel so LUCKY to have found this used (phone...
1    nice phone, nice up grade from my pantach revu...
2                                         Very pleased
3    It works good but it goes slow sometimes but i...
4    Great phone to replace my lost phone. The only...
Name: Reviews, dtype: object

In [14]:
# viewing the first five reviews as text
for text in original['Reviews'].head(5):
    print(text)
    print('\n')

I feel so LUCKY to have found this used (phone to us & not used hard at all), phone on line from someone who upgraded and sold this one. My Son liked his old one that finally fell apart after 2.5+ years and didn't want an upgrade!! Thank you Seller, we really appreciate it & your honesty re: said used phone.I recommend this seller very highly & would but from them again!!


nice phone, nice up grade from my pantach revue. Very clean set up and easy set up. never had an android phone but they are fantastic to say the least. perfect size for surfing and social media. great phone samsung


Very pleased


It works good but it goes slow sometimes but its a very good phone I love it


Great phone to replace my lost phone. The only thing is the volume up button does not work, but I can still go into settings to adjust. Other than that, it does the job until I am eligible to upgrade my phone again.Thaanks!




In [1]:
# removing all non characters and replacing spaces with underscore
def clean_and_concatenate(col):
    return [re.sub(pattern="\W+", repl="", 
                  string=str(x).replace(" ", "_").lower()) for x in col]

In [16]:
# removing all non characters except space
def clean_and_remove_punctuations(col):
    stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", 
                  "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 
                  'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 
                  'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 
                  'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 
                  'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 
                  'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 
                  'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 
                  'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 
                  'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 
                  's', 't', 'can', 'will', 'just', 'don', 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 
                  've', 'y', 'ain', 'aren', 'ma']
    lemmatizer = WordNetLemmatizer() 
    lst = []
    for x in col:
        row = re.sub(r"[^a-zA-Z0-9]+", ' ', x).lower().split()
        lst += [" ".join([lemmatizer.lemmatize(x) for x in row if x not in stop_words])]
    return lst

In [17]:
# modifying the original dataframe and creating a new dataframe
modified_df = original.copy()
modified_df['Reviews'] = clean_and_remove_punctuations(modified_df['Reviews'])

In [18]:
# printing the shape and first ten rows of the modified dataframe
print(modified_df.shape)
modified_df.head(10)

(413778, 2)


,Rating,Reviews
0,5,feel lucky found used phone u not used hard ph...
1,4,nice phone nice grade pantach revue clean set ...
2,5,pleased
3,4,work good go slow sometimes good phone love
4,4,great phone replace lost phone thing volume bu...
5,1,already phone problem know stated used dang no...
6,2,charging port loose got soldered needed new ba...
7,2,phone look good wouldn stay charged buy new ba...
8,5,originally using samsung s2 galaxy sprint want...
9,3,battery life great responsive touch issue some...


In [19]:
# renaming the columns
modified_df.columns = ['rating', 'review']
modified_df.head()

,rating,review
0,5,feel lucky found used phone u not used hard ph...
1,4,nice phone nice grade pantach revue clean set ...
2,5,pleased
3,4,work good go slow sometimes good phone love
4,4,great phone replace lost phone thing volume bu...


In [20]:
# saving the dataframe to a csv file for further processing
modified_df.to_csv("Cleaned_mobile_unlocked.csv")